# Sweeps notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pprint
import os
import re
import json
import sys
import wandb
from model import *
from pytorch_lightning.loggers import WandbLogger
import tensorflow

c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
# Define sweep config
sweep_config = {
    'method': 'random' # random, grid, or bayes
    }

In [3]:
# Define metric
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   # minimize or maximize
    }
# add in sweep_config
sweep_config['metric'] = metric

In [4]:
epochs = 30
architecture = 'CNN'
dataset = 'Poems'
# Define hyperparameter space
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'embedding_dim': {
        'values': [128, 256, 512]
        },
    }
# we can indicate the distribution for continuous variables
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.1
      }
    })
# we set values that we want to track but don't want to change, just indicate 1 value
parameters_dict.update({
    'epochs': {'value': epochs},
    "architecture":{'value': architecture},
    "dataset": {'value': dataset},        
    })
# add params in sweep_config
sweep_config['parameters'] = parameters_dict

In [5]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'architecture': {'value': 'CNN'},
                'dataset': {'value': 'Poems'},
                'embedding_dim': {'values': [128, 256, 512]},
                'epochs': {'value': 30},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [6]:
# Log in to weights and biases
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jjdsantiago3 (msds_mlops2023_lt2). Use `wandb login --relogin` to force relogin


True

In [7]:
sweep_id = wandb.sweep(sweep_config, project="mlo-final-project")

Create sweep with ID: w94pa8s6
Sweep URL: https://wandb.ai/msds_mlops2023_lt2/mlo-final-project/sweeps/w94pa8s6


In [8]:
# define hyperparam search function
def hyperparam_search(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        # use config params
        learning_rate = config.learning_rate
        optimizer = config.optimizer
        embedding_dim = config.embedding_dim
        epochs = config.epochs

        # define model
        _model = poem_classifier_model()

        _model.load_data()
        _model.preprocess()
        _model.train(embd_dim=embedding_dim, epochs=epochs, lr=learning_rate, optimizer=optimizer)

        # print(_model.trained_model.history)

        for i in range(epochs):
            wandb.log({"train_loss": _model.trained_model.history['loss'][i],
                       "train_acc": _model.trained_model.history['acc'][i],
                       "val_loss": _model.trained_model.history['val_loss'][i], 
                       "val_acc": _model.trained_model.history['val_acc'][i]})
        
        results = _model.test()
        wandb.log({"test_loss": results[0], "test_acc": results[1]})

wandb.agent(sweep_id, hyperparam_search, count=10) # count - num iters

wandb: Agent Starting Run: oia9cawn with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 256
wandb: 	epochs: 30
wandb: 	learning_rate: 0.009654640739687003
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

Epoch 1/30
21/21 [==============================] - 1s 19ms/step - loss: 1.3558 - acc: 0.3109 - val_loss: 1.3690 - val_acc: 0.2917
Epoch 2/30
21/21 [==============================] - 0s 11ms/step - loss: 1.2712 - acc: 0.4469 - val_loss: 1.3217 - val_acc: 0.3810
Epoch 3/30
21/21 [==============================] - 0s 11ms/step - loss: 1.1378 - acc: 0.5217 - val_loss: 1.3140 - val_acc: 0.4107
Epoch 4/30
21/21 [==============================] - 0s 11ms/step - loss: 0.9841 - acc: 0.6203 - val_loss: 1.4095 - val_acc: 0.3810
Epoch 5/30
21/21 [==============================] - 0s 11ms/step - loss: 0.8748 - acc: 0.6622 - val_loss: 1.3147 - val_acc: 0.4048
Epoch 6/30
21/21 [==============================] - 0s 11ms/step - loss: 0.7654 - acc: 0.7175 - val_loss: 1.4589 - val_acc: 0.4107
Epoch 7/30
21/21 [==============================] - 0s 11ms/step - loss: 0.6757 - acc: 0.7608 - val_loss: 1.4470 - val_acc: 0.3810
Epoch 8/30
21/21 [==============================] - 0s 12ms/step - loss: 0.6082 - a

test_acc,▁
test_loss,▁
train_acc,▁▃▃▄▅▆▆▆▇▇▇▇▇▇██▇█████████████
train_loss,█▇▇▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆█▆██▆▆▆▅▅▆▆▆▅▆▆▆▄▇▅▆▆▆▅▄▅▆▆▅
val_loss,▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇█████
test_acc,0.33333
test_loss,2.91291
train_acc,0.92676
train_loss,0.18655
val_acc,0.35714


wandb: Agent Starting Run: e0m21s3t with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 512
wandb: 	epochs: 30
wandb: 	learning_rate: 0.02808494175379332
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

test_acc,▁
test_loss,▁
train_acc,▂▃▄▃▂▂▃▁▅▆▇▄▆▅▆▅▄▅▅▃▃▅▄▅▄▅▅█▇▄
train_loss,█▆▅▄▄▄▄▄▃▃▃▃▃▃▂▃▂▂▃▂▂▂▂▂▂▂▂▁▁▂
val_acc,█▇▇▆▆▁▇▅▆▄▄▆▅▇▇▅▅█▆▇▅▄▅▆▅▇▅▅█▇
val_loss,█▅▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▂▂▁▂▁▁▂▁▁
test_acc,0.14
test_loss,1.60886
train_acc,0.29447
train_loss,1.36101
val_acc,0.30952


wandb: Agent Starting Run: e1tet2j1 with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 512
wandb: 	epochs: 30
wandb: 	learning_rate: 0.02439222145389936
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/30


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

21/21 [==============================] - 1s 28ms/step - loss: 1.3669 - acc: 0.3333 - val_loss: 1.4990 - val_acc: 0.3036
Epoch 2/30
21/21 [==============================] - 0s 22ms/step - loss: 1.1602 - acc: 0.4828 - val_loss: 1.3042 - val_acc: 0.3571
Epoch 3/30
21/21 [==============================] - 0s 22ms/step - loss: 0.8827 - acc: 0.6607 - val_loss: 1.5049 - val_acc: 0.3750
Epoch 4/30
21/21 [==============================] - 0s 22ms/step - loss: 0.7489 - acc: 0.7115 - val_loss: 1.6263 - val_acc: 0.3452
Epoch 5/30
21/21 [==============================] - 0s 22ms/step - loss: 0.6307 - acc: 0.7638 - val_loss: 1.8153 - val_acc: 0.3810
Epoch 6/30
21/21 [==============================] - 0s 21ms/step - loss: 0.5764 - acc: 0.7877 - val_loss: 2.0118 - val_acc: 0.3452
Epoch 7/30
21/21 [==============================] - 0s 20ms/step - loss: 0.5237 - acc: 0.8087 - val_loss: 2.0693 - val_acc: 0.3452
Epoch 8/30
21/21 [==============================] - 0s 21ms/step - loss: 0.4586 - acc: 0.8311 

In [ ]:
wandb.finish()